### Imports

In [32]:
import os
import json
import requests
from bs4 import BeautifulSoup

### Excraction of opinions

In [29]:
with open("./cookie.json", "r", encoding="UTF-8") as jf: 
	headers = json.load(jf)

In [30]:
product_id = "84514582"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"
all_opinions = []
while url:
    response = requests.get(url, headers = headers)
    if response.status_code == 200:
        print(url)
        page_dom = BeautifulSoup(response.text, 'html.parser')
        opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")
        print(len(opinions))
        for opinion in opinions:
            single_opinion = {}
            single_opinion['opinion_id'] = opinion["data-entry-id"]
            single_opinion['author'] = opinion.select_one("span.user-post__author-name").text.strip()
            try:
                single_opinion['recommend'] = opinion.select_one("span.user-post__author-recomendation > em").text.strip()
            except AttributeError:
                single_opinion['recommend'] = None
            single_opinion['stars'] = opinion.select_one("span.user-post__score-count").text.strip()
            single_opinion['content'] = opinion.select_one("div.user-post__text").text.strip()
            single_opinion['pros'] = [tag.text.strip() for tag in opinion.select("div.review-feature__item--positive")]
            single_opinion['cons'] = [tag.text.strip() for tag in opinion.select("div.review-feature__item--negative")]
            single_opinion['up_votes'] = opinion.select_one("button.vote-yes")["data-total-vote"].strip()
            single_opinion['down_votes'] = opinion.select_one("button.vote-no")["data-total-vote"].strip()
            single_opinion['published'] = opinion.select_one("span.user-post__published > time:nth-child(1)")["datetime"].strip()
            try:
                single_opinion['purchased'] = opinion.select_one("span.user-post__published > time:nth-child(2)")["datetime"].strip()
            except TypeError:
                single_opinion['purchased'] = None
            all_opinions.append(single_opinion)
        try:
            url = "https://www.ceneo.pl"+page_dom.select_one("a.pagination__next")["href"]
        except TypeError:
            url = None

https://www.ceneo.pl/84514582#tab=reviews
10
https://www.ceneo.pl/84514582/opinie-2
10
https://www.ceneo.pl/84514582/opinie-3
10
https://www.ceneo.pl/84514582/opinie-4
10
https://www.ceneo.pl/84514582/opinie-5
10
https://www.ceneo.pl/84514582/opinie-6
10
https://www.ceneo.pl/84514582/opinie-7
0


### Creation of database

In [33]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")

In [36]:
with open(f"./opinions/{product_id}.json", "w", encoding="UTF-8") as jf:
    json.dump(all_opinions, jf, indent=4, ensure_ascii=False)